### 目的条件
- binaryファイルをorderのパラメータに基づきプロット周波数プロットできるようにする
- 一旦波形を(2,2)のサブプロットを用いて[波形(全体),波形(0-500us), スペクトル(100Mhzまで), スペクトル(3000Hzまで)]の形で格納する

In [7]:
# 公開ライブラリの読み込み
import pandas as pd
import numpy as np
import json
import os, sys
import scipy.io
from scipy import signal
import matplotlib.pyplot as plt
import statistics
import math
import gc
import sys
import os
import warnings
from pathlib import Path
import re
import json
from collections import defaultdict

# 独自ライブラリの読み込み
import convert_bin_to_csv

In [29]:
#フォルダとorderの対応付け
dir_path = Path("E:/experiment/20220310_AE_SENSOR/bin")
data_dirs= list(dir_path.glob("*"))
order_path = Path("E:/experiment/20220310_AE_SENSOR/order")
order_files= list(order_path.glob("*"))
data_order_dic=dict(zip(data_dirs,order_files))
data_order_dic

{WindowsPath('E:/experiment/20220310_AE_SENSOR/bin/01_us_burst'): WindowsPath('E:/experiment/20220310_AE_SENSOR/order/220310_1736_order.csv'),
 WindowsPath('E:/experiment/20220310_AE_SENSOR/bin/02_sham'): WindowsPath('E:/experiment/20220310_AE_SENSOR/order/220310_1747_order.csv'),
 WindowsPath('E:/experiment/20220310_AE_SENSOR/bin/03_us_cont'): WindowsPath('E:/experiment/20220310_AE_SENSOR/order/220310_1758_order.csv'),
 WindowsPath('E:/experiment/20220310_AE_SENSOR/bin/04_cont_sham'): WindowsPath('E:/experiment/20220310_AE_SENSOR/order/220310_1812_order.csv'),
 WindowsPath('E:/experiment/20220310_AE_SENSOR/bin/05_low_amp_cont'): WindowsPath('E:/experiment/20220310_AE_SENSOR/order/220310_1848_order.csv'),
 WindowsPath('E:/experiment/20220310_AE_SENSOR/bin/06_low_amp_burst'): WindowsPath('E:/experiment/20220310_AE_SENSOR/order/220310_1920_order.csv'),
 WindowsPath('E:/experiment/20220310_AE_SENSOR/bin/07_burst_37mm'): WindowsPath('E:/experiment/20220310_AE_SENSOR/order/220310_2004_order

In [3]:
single_dir=list(data_order_dic.keys())[0]
order_file_path = data_order_dic[single_dir]


In [41]:
bin_files=list(single_dir.glob("*"))

In [37]:
#fileの末尾が(2)など1文字だと面倒なのでrenameする処理をする
target_file=bin_files[1]
target_file.name
num=re.search("[0-9]+",target_file.name).group(0)
os.rename(target_file,f"{target_file.parent}\{target_file.name.replace(num,str(num).zfill(2))}")

'usr_wf_data (1).bin'

In [42]:
#file名を0埋め
for bin_file in bin_files:
    try:
        num=re.search("[0-9]+",bin_file.name).group(0)
        os.rename(bin_file,f"{bin_file.parent}\{bin_file.name.replace(num,str(num).zfill(2))}")
    except:
        num="(00)"
        os.rename(bin_file,f"{bin_file.parent}\{bin_file.name.replace('.bin',' '+num+'.bin')}")

In [43]:
for dir in data_dirs:
    bin_files=list(dir.glob("*"))
    #file名を0埋め
    for bin_file in bin_files:
        try:
            num=re.search("[0-9]+",bin_file.name).group(0)
            os.rename(bin_file,f"{bin_file.parent}\{bin_file.name.replace(num,str(num).zfill(2))}")
        except:
            num="(00)"
            os.rename(bin_file,f"{bin_file.parent}\{bin_file.name.replace('.bin',' '+num+'.bin')}")

In [35]:
def plot_wave(axes,index,time_datas,volt_datas,xlim,title):
    row=index[0]
    col=index[1]
    ax=axes[row][col]
    time_datas=np.array(time_datas)
    ax.plot(time_datas,volt_datas)
    ax.set_xlim(xlim)
    if max(volt_datas)<=0.5:
        ax.set_ylim[-0.5,0.5]
    ax.set_xlabel("time from stimulation[ms]")
    ax.set_ylabel("voltage [V]")
    ax.set_title(title)

In [36]:
def plot_fft(axes,index,volt_datas,samplerate,xlim,title):
    #振幅スペクトルを描画する
    row=index[0]
    col=index[1]
    ax=axes[row][col]
    point=len(volt_datas)
    d=1/samplerate
    point=len(volt_datas)
    F=np.fft.fft(volt_datas,n=point)
    freq=np.fft.fftfreq(n=point,d=d)
    Amp=np.abs(F/(point/2))
    left_point=np.argmin(freq[freq>=xlim[0]])
    right_point=np.argmax(freq[freq<=xlim[1]])
    xlim_point=[left_point,right_point]
    ax.plot(freq[xlim_point[0]:xlim_point[1]], Amp[xlim_point[0]:xlim_point[1]])
    ax.set_xlabel("Freqency [Hz]")
    ax.set_ylabel("Amplitude")
    ax.set_title(title)
    

In [30]:
def plot_and_fft(data_dir,order_table):
    """_summary_

    Args:
        data_dir (Path): _description_
        order_table (Pandas.Dataframe): _description_
    """
    #　各波形データとorder_tableの対応付け
    index=0
    if not(os.path.exists("./waveplot")):
        os.mkdir("./waveplot")
    if not(os.path.exists(f"./waveplot/{data_dir.name}")):
        os.mkdir(f"./waveplot/{data_dir.name}")
    dir_path=f"./waveplot/{data_dir.name}" 
    bin_files=list(data_dir.glob("*"))
    counts=defaultdict(int)
    for bin_file in bin_files:
        line=order_table.iloc[index]
        if line["state"]=="click":
            title=f'{line["db"]}_{line["trial"]}'
        elif line["state"]=="us_burst":
            title=f'{line["amp"]*1000}mV_{line["duration"]*1000}ms_{line["pulse_duration"]*1000000}us_PRF{line["PRF"]}Hz_window{line["window"]}%'
        elif line["state"]=="us_cont":
            title=f'{line["amp"]*1000}mV_{line["duration"]*1000}ms_window{int(line["window"]*1000)}ms'
            
        if title in counts:
            counts[title]+=1
        else:
            counts[title]=0
            
        title=f"{title}_{counts[title]}"
        
        time_volt_data=convert_bin_to_csv.bin_to_csv(bin_file,f"{title}.csv",str(data_dir.name))
        print(time_volt_data[:10])
        time_volt_data=time_volt_data[0]
        time_datas=[record[0] for record in time_volt_data]
        volt_datas=[record[1] for record in time_volt_data]
        wide_xlim=[-10,120] #[ms]
        narrow_xlim=[0,0.5] #[ms]
        fig=plt.figure(tight_layout=True)
        #波形データをrow=0にfftデータをrow=1に保存する
        axes = fig.subplots(2, 2)
        samplerate=2.5*10**7
        plot_wave(axes,[0,0],time_datas,volt_datas,wide_xlim,"whole voltage")
        plot_wave(axes,[0,1],time_datas,volt_datas,narrow_xlim,f"patial voltage {narrow_xlim[0]}ms-{narrow_xlim[1]}ms")
        plot_fft(axes,[1,0],volt_datas,samplerate,[0,1000000],f"amp spectal 1-1000000Hz")
        plot_fft(axes,[1,1],volt_datas,samplerate,[0,10000], f"amp spectal 1-10000Hz")
        plt.savefig(f"{dir_path}/{title}.png")
        index+=1
        

In [31]:
def convert_files(data_dir,order_table):
    """_summary_

    Args:
        data_dir (Path): _description_
        order_table (Pandas.Dataframe): _description_
    """
    #　各波形データとorder_tableの対応付け
    index=0
    if not(os.path.exists("./waveplot")):
        os.mkdir("./waveplot")
    if not(os.path.exists(f"./waveplot/{data_dir.name}")):
        os.mkdir(f"./waveplot/{data_dir.name}")
    dir_path=f"./waveplot/{data_dir.name}" 
    bin_files=list(data_dir.glob("*"))
    counts=defaultdict(int)
    for bin_file in bin_files:
        line=order_table.iloc[index]
        if line["state"]=="click":
            title=f'{line["db"]}'
        elif line["state"]=="us_burst":
            pulse_duration=str(round(line["pulse_duration"],3))
            title=f'{line["amp"]*1000}mV_{line["duration"]*1000}ms_PD_{pulse_duration}us_PRF_{line["PRF"]}Hz_window_{line["window"]}%'
        elif line["state"]=="us_cont":
            title=f'{line["amp"]*1000}mV_{line["duration"]*1000}ms_window_{int(line["window"]*1000)}ms'
            
        if title in counts:
            counts[title]+=1
        else:
            counts[title]=0
            
        title=f"{title}_{counts[title]}"
        print(title)        
        time_volt_data=convert_bin_to_csv.bin_to_csv(bin_file,f"{title}.csv",str(data_dir.name))
        index+=1
        del time_volt_data
        gc.collect()

In [12]:
data_order_dic=dict(zip(list(data_order_dic.keys())[10:],list(data_order_dic.values())[10:]))
data_order_dic

NameError: name 'data_order_dic' is not defined

In [33]:

for data_dir,order_file in data_order_dic.items():
    #与えた刺激のデータを読み込み
    order_table = pd.read_csv(order_file)
    #読み込んだorder_tableとdirを元に波形を描画する
    convert_files(data_dir,order_table)

50.0_0
7000000
0
Read data from bin file finished!
Writing to csv head...
start to block
BLOCK0 range(0, 1000000) converting...
analog converting...
BLOCK0 writing to csv...
BLOCK1 range(1000000, 2000000) converting...
analog converting...
BLOCK1 writing to csv...
BLOCK2 range(2000000, 3000000) converting...
analog converting...
BLOCK2 writing to csv...
BLOCK3 range(3000000, 4000000) converting...
analog converting...
BLOCK3 writing to csv...
BLOCK4 range(4000000, 5000000) converting...
analog converting...
BLOCK4 writing to csv...
BLOCK5 range(5000000, 6000000) converting...
analog converting...
BLOCK5 writing to csv...
BLOCK6 range(6000000, 7000000) converting...
analog converting...
BLOCK6 writing to csv...
80.0_0
7000000
0
Read data from bin file finished!
Writing to csv head...
start to block
BLOCK0 range(0, 1000000) converting...
analog converting...
BLOCK0 writing to csv...
BLOCK1 range(1000000, 2000000) converting...
analog converting...
BLOCK1 writing to csv...
BLOCK2 range(20

In [32]:
def load_wave(file_path):
    #各CSVファイルを読み込んで波形と時間データを出力
    data = pd.read_csv(file_path,header=6,names=["Source","CH1","CH2"])
    time=data["Source"].values
    time=np.array(list(map(float,time)))*1000
    voltage=data["CH1"].values
    voltage=np.array(list(map(float,voltage)))
    return time,voltage


In [33]:
csv_dirpath=Path("./csv")
csv_dirs=list(csv_dirpath.glob("*"))
csv_dirs


[WindowsPath('csv/01_us_burst'),
 WindowsPath('csv/02_sham'),
 WindowsPath('csv/03_us_cont'),
 WindowsPath('csv/04_cont_sham'),
 WindowsPath('csv/05_low_amp_cont'),
 WindowsPath('csv/06_low_amp_burst'),
 WindowsPath('csv/07_burst_37mm'),
 WindowsPath('csv/08_burst_42mm'),
 WindowsPath('csv/09_no_gel_burst'),
 WindowsPath('csv/10_no_gel_cont'),
 WindowsPath('csv/11_click')]

In [34]:
samplingrate=2.5*10**7
for csv_dir in csv_dirs:
    dir_name=csv_dir.name
    csv_files=list(csv_dir.glob("*"))
    for csv_file in csv_files:
        print(f"loading {dir_name} {csv_file.name}")
        time,vol_wave=load_wave(csv_file)
        title=csv_file.name[:-4]
        fig=plt.figure(tight_layout=True)
        #波形データをrow=0にfftデータをrow=1に保存する
        axes = fig.subplots(2, 2)
        samplerate=2.5*10**7
        wide_xlim=[-10,120]
        narrow_xlim=[0,0.5]
        if not(os.path.exists(f"./waveplot/{dir_name}")):
            os.mkdir(f"./waveplot/{dir_name}")
        plot_wave(axes,[0,0],time,vol_wave,wide_xlim,"whole voltage")
        plot_wave(axes,[0,1],time,vol_wave,narrow_xlim,f"patial voltage {narrow_xlim[0]}ms-{narrow_xlim[1]}ms")
        plot_fft(axes,[1,0],vol_wave,samplerate,[0,1000000],f"amp spectal 1-1000000Hz")
        plot_fft(axes,[1,1],vol_wave,samplerate,[0,5000], f"amp spectal 1-5000Hz")
        plt.savefig(f"./waveplot/{dir_name}/{title}.png")
        plt.cla()
        plt.clf()
        plt.close(fig)
        del axes,time,vol_wave,fig
        gc.collect()
        
        

loading 01_us_burst 1580.0mV_100.0ms_160.0us_PRF1500.0Hz_window0.0%_0.csv
loading 01_us_burst 1580.0mV_100.0ms_160.0us_PRF1500.0Hz_window0.0%_1.csv
loading 01_us_burst 1580.0mV_100.0ms_160.0us_PRF1500.0Hz_window0.0%_2.csv
loading 01_us_burst 1580.0mV_100.0ms_160.0us_PRF1500.0Hz_window25.0%_0.csv
loading 01_us_burst 1580.0mV_100.0ms_160.0us_PRF1500.0Hz_window25.0%_1.csv
loading 01_us_burst 190.0mV_100.0ms_160.0us_PRF1500.0Hz_window0.0%_0.csv
loading 01_us_burst 190.0mV_100.0ms_160.0us_PRF1500.0Hz_window0.0%_1.csv
loading 01_us_burst 190.0mV_100.0ms_160.0us_PRF1500.0Hz_window0.0%_2.csv
loading 01_us_burst 190.0mV_100.0ms_160.0us_PRF1500.0Hz_window25.0%_0.csv
loading 01_us_burst 190.0mV_100.0ms_160.0us_PRF1500.0Hz_window25.0%_1.csv
loading 01_us_burst 190.0mV_100.0ms_160.0us_PRF1500.0Hz_window25.0%_2.csv
loading 01_us_burst 2370.0mV_100.0ms_160.0us_PRF1500.0Hz_window0.0%_0.csv
loading 01_us_burst 2370.0mV_100.0ms_160.0us_PRF1500.0Hz_window0.0%_1.csv
loading 01_us_burst 2370.0mV_100.0ms_16